# 第一章 数据结构与算法

## 1.11 对切片命名

问题：面对硬编码切片索引无法阅读，想理清楚他们。

解决方案： 假设有一些代码用来从字符串的固定位置中取出具体的数据，对他们命名：

注：如果代码中有很多硬编码的索引值，可读性和可维护性都不好。

In [ ]:
SHARES = slice(20, 32)
PRICE = slice(40,48)

cost = int(record[SHARES]) * float(recordd[PRICE])

一般来说，内置的slice()函数会创建一个切片对象，可以用在任何允许进行切片操作的地方。例如：

In [3]:
items = [0, 1, 2, 3, 4, 5, 6]
a = slice(2, 4)
print("items[2:4]:  ", items[2:4])
print("items[a]:    ", items[a])
items[a] = [7, 7, 7]
print("items:", items)

items[2:4]:   [2, 3]
items[a]:     [2, 3]
items: [0, 1, 7, 7, 7, 4, 5, 6]


如果有一个slice对象的实例s，可以分别通过s.start、s.stop以及s.step属性获得该对象的信息。

In [6]:
s = slice(3,7,2)
print('s.start:',s.start)
print('s.step: ',s.step)

s.start: 3
s.step:  2


 还可以通过indices(size)方法将切片映射到特定大小的序列上。这会返回一个（start,stop,step）元组，所有制都已经恰当地限制在边界以内，当做索引操作的时候可避免出现IndexError异常。


## 1.12 找出序列中出现次数最多的元素

解决方案： `collections`模块中的`Counter`类是为此类问题设计的。其中的`most_common()`方法可以解决此问题。

In [16]:
from collections import Counter

words = ['a', 'a', 'a', 'b', 'b','b','b','c','c','d']
word_counts = Counter(words)
top_three = word_counts.most_common(3)
print('top 3:', top_three)

top 3: [('b', 4), ('a', 3), ('c', 2)]


可给`Counter`对象提供任何可hash的对象序列作为输入。底层实现中，`Counter`是一个**字典**，在元素和它们出现次数之间做了映射。

In [18]:
word_counts['a']

3

若想用word_counts同时统计words2，可以通过以下两种方法：

In [19]:
words2 = ['a', 'a', 'a', 'd','e']

In [20]:
# 方法一
for word in words2:
    word_counts[word] +=1
    
# 方法二
word_counts.update(words2)

In [21]:
word_counts

Counter({'a': 9, 'b': 4, 'c': 2, 'd': 3, 'e': 2})

In [23]:
word_counts2 = Counter(words)
word_counts2

Counter({'a': 3, 'b': 4, 'c': 2, 'd': 1})

`Counter`对象可进行各种数学运算

In [24]:
word_counts - word_counts2

Counter({'a': 6, 'd': 2, 'e': 2})

In [25]:
word_counts + word_counts2

Counter({'a': 12, 'b': 8, 'c': 4, 'd': 4, 'e': 2})

## 1.13 通过公共键对字典列表排序

问题：对于已有的**字典**列表，想根据一个或多个字典中的值对列表排序。

解决方案：利用`operator`模块中的`itemgetter`函数对这类结构进行排序。例如我们有以下数据结构：


In [28]:
rows = [
    {'fname':'aaa','lname':'qqq','uid':444},
    {'fname':'ccc','lname':'eee','uid':333},
    {'fname':'ddd','lname':'rrr','uid':222},
    {'fname':'eee','lname':'ppp','uid':111}
]

根据所有字典中**共有的字段**对其进行排序：


In [29]:
from operator import itemgetter

rows_by_fname = sorted(rows, key = itemgetter('fname'))
rows_by_uid = sorted(rows, key = itemgetter('uid'))

print("fname:  ", rows_by_fname)
print("uid:    ", rows_by_uid)

fname:   [{'fname': 'aaa', 'lname': 'qqq', 'uid': 444}, {'fname': 'ccc', 'lname': 'eee', 'uid': 333}, {'fname': 'ddd', 'lname': 'rrr', 'uid': 222}, {'fname': 'eee', 'lname': 'ppp', 'uid': 111}]
uid:     [{'fname': 'eee', 'lname': 'ppp', 'uid': 111}, {'fname': 'ddd', 'lname': 'rrr', 'uid': 222}, {'fname': 'ccc', 'lname': 'eee', 'uid': 333}, {'fname': 'aaa', 'lname': 'qqq', 'uid': 444}]


`itemgetter()`函数还可以接受多个键。如：

In [30]:
rows_by_lfname = sorted(rows, key=itemgetter('lname','fname'))
print("lfname:   ", rows_by_lfname)

lfname:    [{'fname': 'ccc', 'lname': 'eee', 'uid': 333}, {'fname': 'eee', 'lname': 'ppp', 'uid': 111}, {'fname': 'aaa', 'lname': 'qqq', 'uid': 444}, {'fname': 'ddd', 'lname': 'rrr', 'uid': 222}]


上例中，rows被传递给内建函数`sorted()`，该函数接受一个关键字参数key。这个参数应该代表一个可调用对象（callable），该对象从rows中接受一个**单独的元素**作为输入并返回一个用来做排序依据的值。`itemgetter()`函数创建的就是这样一个可调用对象。


函数`operator.itemgetter()`接受的参数可作为查询的标记，用来从rows的记录中提取出所需的值。它可以是任何可以传递个对象的`\_\_getitem()\_\_`方法的值。如果传递多个标记给`itemgetter()`，那么它产生的可调用对象将返回一个包含所有元素在内的元组。

有时可用**lambda**表达式取代`itemgetter()`的功能(用`itemgetter()`的方法会运行的更快一点）：

In [ ]:
rows_by_fname = sorted(rows, key = lambda r: r['fname'])
rows_by_lfname = sorted(rows, key = lambda r: (r['lname'], r['fname']))

## 1.14 对不原生支持比较操作的对象排序

问题：想在同一个**类**的实例之间做排序，但它们不原生支持比较操作。

解决方案：内建的`sorted()`函数可接受一个用来传递可调用对象（callable）的参数key，而该可调用对象会返回待排序对象中的某些值，sorted利用这些值来比较对象。例如：

In [35]:
class User:
    def __init__(self, user_id):
        self.user_id = user_id
    
    def __repr__(self):
        return 'User({})'.format(self.user_id)    

In [36]:
# 想通过user_id属性对他们的实例进行排序
user = [User(23), User(3), User(4)]
user

[User(23), User(3), User(4)]

In [37]:
sorted(user, key = lambda x: x.user_id)

[User(3), User(4), User(23)]

或使用`operator.attrgetter('user_id')`

In [38]:
from operator import attrgetter

In [41]:
sorted(user, key =attrgetter('user_id'))

[User(3), User(4), User(23)]

## 1.15 根据字段将记录分组

问题： 有一系列**字典或对象实例**，想根据某个特定的字段来分组迭代数据。

解决方案：`itertools.groupby()`函数在对数据进行分组时特别有用。如下：

In [52]:
# 假设有如下字典列表
rows = [
    {'address':'1111','date':'d7777'},
    {'address':'2222','date':'d6666'},
    {'address':'3333','date':'d5555'},
    {'address':'4444','date':'d4444'},
    {'address':'5555','date':'d3333'},
    {'address':'6666','date':'d1111'},
    {'address':'7777','date':'d1111'}
]

假设想根据日期以分组的方式迭代数据。首先以目标字段（如上中的date）对序列排序，然后使用`itertools.groupby()`。

In [53]:
from operator import itemgetter
from itertools import groupby

# sort by the desired field first
rows.sort(key=itemgetter('date'))

# iterate in groups
for date, items in groupby(rows, key = itemgetter('date')):
    print("date:  ", date)
    print("items: ", items)
    
    for i in items:
        print('|| ', i)

date:   d1111
items:  <itertools._grouper object at 0x000001EA85CA35C8>
||  {'address': '6666', 'date': 'd1111'}
||  {'address': '7777', 'date': 'd1111'}
date:   d3333
items:  <itertools._grouper object at 0x000001EA85C1B9C8>
||  {'address': '5555', 'date': 'd3333'}
date:   d4444
items:  <itertools._grouper object at 0x000001EA85AAB988>
||  {'address': '4444', 'date': 'd4444'}
date:   d5555
items:  <itertools._grouper object at 0x000001EA85C1B9C8>
||  {'address': '3333', 'date': 'd5555'}
date:   d6666
items:  <itertools._grouper object at 0x000001EA85AABE08>
||  {'address': '2222', 'date': 'd6666'}
date:   d7777
items:  <itertools._grouper object at 0x000001EA85C1B9C8>
||  {'address': '1111', 'date': 'd7777'}


`groupby()`函数通过扫描序列找出拥有相同值（或由参数key指定的函数所返回的值）的序列项，并将其分组。

`groupby()`函数创建了一个迭代器，每次迭代时都会返回一个值(value)和一个子迭代器(sub_iterator)，这个子迭代器可以产生所有在该分组内具有该值的项。

对于上例来说，`date`是返回的值，`item`是返回的sub_iterator，其包含了`date`为相同值的项。

注意：`groupby()`只能检查连续的项，所有需要先根据感兴趣的字段对数据进行排序，如果不排序，将无法按所想的方式对记录分组。

如果只想简单地根据日期将数据分组到一起，放进一个大的数据结构中以允许随机访问，那么利用`defaultdict()`构建一个一键多值的字典(multidict)可能会更好，如：


In [54]:
from collections import defaultdict
rows_by_date = defaultdict(list)
for row in rows:
    rows_by_date[row['date']].append(row)

In [55]:
rows_by_date

defaultdict(list,
            {'d1111': [{'address': '6666', 'date': 'd1111'},
              {'address': '7777', 'date': 'd1111'}],
             'd3333': [{'address': '5555', 'date': 'd3333'}],
             'd4444': [{'address': '4444', 'date': 'd4444'}],
             'd5555': [{'address': '3333', 'date': 'd5555'}],
             'd6666': [{'address': '2222', 'date': 'd6666'}],
             'd7777': [{'address': '1111', 'date': 'd7777'}]})

如果不考虑内存方面的因素，这种方式比先排序再`groupby()`迭代要来的更快。

## 1.16 筛选序列中的元素

问题：序列中含有一些数据，我们需要提取出其中的值或根据某些标准对序列做删减。

解决方案：筛选数据最简单的方法就是使用列表推导式（list comprehension）。如：


In [56]:
my_list = [1,3,-3,4,5,-9]
[n for n in my_list if n>0]

[1, 3, 4, 5]

使用列表推导式的潜在缺点是如果原始输入非常大，这么做可能会产生一个庞大的结果。可以使用生成器表达式通过迭代的方式筛选结果。如：


In [57]:
pos = (n for n in my_list if n>0)


In [58]:
pos

<generator object <genexpr> at 0x000001EA856F8448>

In [59]:
for x in pos:
    print(x)

1
3
4
5


如果筛选的标准没办法简单地表示在列表推导式或者生成器表达式中，如筛选过程设计异常处理或其他一些复杂的细节，可以将处理筛选逻辑的代码放到单独的函数中，然后使用内建的filter()函数处理。如：

`filter()`创建了一个迭代器。

In [60]:
values = ['1', '2', '4', '-2', 'N/A']
def is_int(val):
    try:
        x = int(val)
        return True
    except ValueError:
        return False
    
ivals = list(filter(is_int, values))
print(ivals)

['1', '2', '4', '-2']


另一个有用的筛选工具`itertools.compress()`，可以把对一个序列的筛选结果是加到另一个相关的序列上。

In [62]:
from itertools import compress
addresses = [
    '111',
    '222',
    '333',
    '444'
]
counts = [0,1,0,3]
# 想构建一个地址列表，其中相应的count值大于2：
mask = [n>2 for n in counts]
list(compress(addresses, mask))

['444']

关键在于创建一个布尔序列`mask`，用来表示哪个元素满足我们的条件。然后`compress()`函数挑选出满足布尔值为True的相应元素。

`compress()`会返回一个迭代器，可以使用`list()`将结果转为列表。

## 从字典提取子集

问题： 想从另一个字典创建一个子集

解决方案：利用字典推导式（dict comprehension）:

In [65]:
prices = {
    'a':11,
    'b':22,
    'c':33,
    'd':44
}
p1 = {key:value for key,value in prices.items() if value>24}
mask= ['a','b']
p2 = {key:value for key,value in prices.items() if key in mask}
print("p1:",p1)
print("p2:",p2)

p1: {'c': 33, 'd': 44}
p2: {'a': 11, 'b': 22}


大部分可以利用字典推导解决的问题也可以通过创建元组序列然后将它们传递给`dict()`函数来完成，如：

注意：字典推导的方案运行更快。

In [67]:
p3 = dict((key, value) for key,value in prices.items() if value>24)
print("p3: ",p3)

p3:  {'c': 33, 'd': 44}


## 1.18 将名称映射到序列的元素中

问题：我们的代码是通过为止（索引，即下标）来访问列表或元组的，这样的问题是有时会使代码变得难以阅读。我们希望可以通过名称来访问元素，减少结构中对位置的依赖性。

解决方案：相比普通元组，`collections.namedtuple()`（命名元组）只增加了**极小的开销**就提供了这些便利。实际上，`collections.namedtuple()`是一个工厂方法，它返回的是Python中标准元组类型的子类。我们为其提供一个类型名称以及相应的字段，它就返回一个可实例化的类、已经定义好的字段传入值等。 例如：


In [68]:
from collections import namedtuple
Subscriber = namedtuple('Subscriber1',['addr','joined'])
sub = Subscriber('as@qq.com','2121')
print(sub)

Subscriber1(addr='as@qq.com', joined='2121')


`namedtuple`的实例看起来就像一个普通的类实例，但它的实例与普通的元组可以互换，而且支持所有普通元组所支持的操作，如索引(indexing)和分解(unpacking)，如：

In [69]:
len(sub)

2

In [70]:
addr, joined = sub
print(f"addr:{addr},sub:{joined}")

addr:as@qq.com,sub:2121


命名元组的作用：将代码同它所控制的元素位置间解耦。

讨论：`namedtuple`的一种可能用法是作为字典的替代，后者需要更多的空间来存储。因此，如果要构建设计字典的大型数据结构，使用`namedtuple`会更加高效。但是同字典不同的是，`namedtuple`是不可变的(immutable)。如：


In [71]:
s = Subscriber('bbb@qq.com','2020')

In [72]:
s.joined = '2021'

AttributeError: can't set attribute

如果想修改任何属性，可以通过使用`namedtuple`实例的`_replace()`方法来实现。该方法会创建一个`全新的命名元组`，并对相应值做替换。如下：

In [75]:
s._replace(joined='2021')

Subscriber1(addr='bbb@qq.com', joined='2021')

## 1.19 同时对数据做转换和换算

问题： 我们需要用一个换算(reduction)函数(如sum()、min()、max())，首先得对数据做转换或筛选。

解决方案：将数据换算和转换结合在一起，在函数参数中使用生成器表达式。如：

In [77]:
# 计算平方和
nums = [1, 2, 3, 4, 5]
s = sum(x * x for x in nums)

基于生成器的解决方案可以以迭代的方式转换数据，因此在内存使用上高效更多。

## 1.20 将多个映射合并为单个映射

问题： 有多个字典或映射，想在逻辑上将它们合并为一个单独的映射结构，以执行特定操作，比如查找值或检查键是否存在。

解决方案：利用`collections`模块中的`ChainMap`类来解决这个问题。如：


In [78]:
# 有两个字典：
a = {'x':1, 'z':3}
b = {'y':2, 'z':4}

from collections import ChainMap
c = ChainMap(a,b)
print(c['x'])
print(c['y'])
print(c['z'])

1
2
3


`ChainMap`可接受多个映射然后在逻辑上使它们表现为一个单独的映射结构。但这些映射在字面上不会合并在一起。相反，`ChainMap`只是简单地维护一个记录底层映射关系的**列表**，然后重新定义常见的字典操作来扫描这个列表。大部分的操作都能正常工作，如：

In [81]:
print('len:',len(c))
print('list(c.keys()):',list(c.keys()))


len: 3
list(c.keys()): ['y', 'z', 'x']


如果有重复的键，那么会采用第一个映射中对应的值。

另一种解决方案：使用字典的`update()`方法将多个字典合并在一起。例如：


In [82]:
merged = dict(b)
merged.update(a)
print(merged)

{'y': 2, 'z': 3, 'x': 1}


注意：`ChainMap`使用的是原始字典，如果通过`ChainMap`改变了键的值，那么在原始字典也会有改变。但后一种解决方法是单独构建了一个完整的字典对象，通过`update()`方法仅改变了新建的这个对象，对原字典不会改变。